In [1]:
import umap.aligned_umap
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
path1 = "/home/yifan/dataset/clean/pairflip/cifar10/0"
path2 = "/home/yifan/Exp/Dropout/0.3/experiment1"
def getData(path,epoch):
    ##### get representation
    X_train = np.load(os.path.join(path,"Model","Epoch_{}".format(epoch),"train_data.npy"))
    X_test = np.load(os.path.join(path,"Model","Epoch_{}".format(epoch),"test_data.npy"))
    
    #### get label
    training_label_loc =  os.path.join(path, "Training_data", "training_dataset_label.pth")
    y_train = torch.load(training_label_loc, map_location="cpu")
    y_train = y_train.numpy()
    testing_label_loc =  os.path.join(path, "Testing_data", "testing_dataset_label.pth") 
    y_test = torch.load(testing_label_loc, map_location="cpu")
    y_test = y_test.numpy()

    return X_train, X_test, y_train, y_test
X_train1, X_test1, y_train1, y_test1 = getData(path1,150)
X_train2, X_test2, y_train2, y_test2 = getData(path2,150)
X_train1 = X_train1.reshape(X_train1.shape[0],X_train1.shape[1])
X_train2 = X_train2.reshape(X_train2.shape[0],X_train2.shape[1])

/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-30 15:24:17.967894: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 15:24:18.713358: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-30 15:24:18.713417: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinf

In [13]:
from scipy.linalg import svd
def orthogonal_mapping(data1, data2):
        """
        use Orthogonal Mapping, map data2 to data1's space。
    
        :param data1: numpy array, shape (n_samples, n_features)
        :param data2: numpy array, shape (n_samples, n_features)
        :return: data2_mapped: numpy array, mapped data2
        """
        # step1: Centralized data
        data1_centered = data1 - np.mean(data1, axis=0)
        data2_centered = data2 - np.mean(data2, axis=0)
        # step2: Calculate the cross-covariance matrix
        C = data2_centered.T @ data1_centered
        # step3: singular value decomposition
        U, _, Vt = svd(C)

        # step4: Compute orthogonal mapping matrix
        W = U @ Vt

        # step5: Apply mapping matrix
        data2_mapped = data2_centered @ W
    
        return data2_mapped

In [14]:
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist
import numpy as np

r1 = X_train1[:5000]
r2 = X_train2[:5000]

r1_normalized = r1
r2_normalized = r2 

from scipy.spatial import procrustes
r2_transformed = orthogonal_mapping(r1_normalized, r2_normalized)
# Assuming r1 and r2 are your two high-dimensional datasets
# with shapes 50000 x 512

# 1. Compute the pairwise distances between r1 and r2
distances = cdist(r1_normalized, r2_transformed, metric='euclidean')

# 2. Linear assignment to find the best matches
# This will give the indices in r1 for each representation in r2
row_indexes, col_indexes = linear_sum_assignment(distances)

# 3. Compute similarity/distances for each pair
similarities = []
for i, j in zip(row_indexes, col_indexes):
    similarity = np.linalg.norm(r1_normalized[i] - r2_transformed[j])
    similarities.append(similarity)

# Now, similarities will contain the distances between matched representations


In [8]:
row_indexes

array([   0,    1,    2, ..., 4997, 4998, 4999])

In [9]:
col_indexes

array([2831,    1,    2, ..., 4997, 4998, 4999])

In [12]:
similarities

[25.798429962602036,
 31.171100376564617,
 21.32719838397886,
 22.366222250414403,
 33.541461207437834,
 27.89417847938703,
 20.38855679224433,
 32.162238227923005,
 29.534870026089774,
 21.074503616886535,
 27.677634322720596,
 23.548808197982908,
 28.40837317221213,
 24.583268063644912,
 20.46474723714276,
 29.549372680623325,
 21.73430124367299,
 20.3659932331471,
 23.74603539591623,
 26.7815106457187,
 18.58683048726732,
 23.452048659931275,
 20.194371038361542,
 30.13109520401219,
 24.076004644165174,
 18.735074099104864,
 32.59216593861456,
 17.62072336875281,
 21.516341576765615,
 20.12325850555301,
 32.24993928068864,
 22.647320756285353,
 34.94340286272646,
 22.37986307765905,
 29.712909557541153,
 20.517513255082697,
 22.9043217597211,
 25.974798484573974,
 19.21561781564253,
 17.393329272310513,
 21.147989385765023,
 21.89679136978069,
 22.241560939199584,
 32.04174701747462,
 20.52591391388625,
 28.550321764778612,
 27.694497179932092,
 25.85038533858925,
 21.80626632179592